## Definição das variaveis globais utilizadas

In [2]:
# Já foram [PETR4_B_0_30min, PETR4_B_0_60min]
dataName    = "PETR4_B_0_60min"            # Name of the data file
setDivision = [0.1, 0.7, 0.2]              # Size of the [optimization, train, test] set
outputName  = "Fechamento"                 # Name of the output variable0

#### Bibliotecas que devem ser instaladas para a execução do trabalho

In [ ]:
%pip install pandas
%pip install scikit-learn
%pip install scipy
%pip install enum
%pip install enum34
%pip install numpy
%pip install matplotlib
%pip install statsmodels
%pip install arch
%pip install tensorflow
%pip install keras-tuner

#### Versões de pacotes utilizadas

In [ ]:
import pandas as pd
import numpy  as np
import sklearn as sk
import matplotlib as mt
import sys

print("Python:  ", sys.version)
print("Pandas:  ", pd.__version__)
print("Numpy:   ", np.__version__)
print("Matplt:  ", mt.__version__)
print("Sklearn: ", sk.__version__)

## Criando a base de dados

#### Métodos  disponíveis
- SMA => Média móvel simples = $\frac{1}{n} \sum_{i=0}^{n-1} Price_{t-x}$ 
    - x => tamanho da janela de amostragem
- EMA => Média móvel exponencial = $\alpha (currentPrice - EMA_{t-1}) + EMA_{t-1}$
    - $\alpha = \frac{2}{n+1}$ => Fator de suavização
    - n => número de amostras na análise
    - $EMA_{t-1}$ => média móvel exponencial anterior
- MACD => Convergência/Divergência da média móvel = $EMA(n) - EMA(k)$
    - n => média exponencial rápida
    - k => média exponencial lenta
- CCI => Índice de Canal de Commodities = $\frac{TP - SMA(TP,N)}{0.015 * DP(TP)}$
    - DP => Desvio padrão de TP
    - SMA => Média móvel simples
    - TP => Preço típico (uma média simples entre os valores de fechamento, alta e baixa)
- ADX => Indicador de força da tendência = $\frac{(n-1)*EMA(TR) + TR}{n}$
    - TR => True Range = $max(H-L, abs(H-C_{t-1}), abs(L-C_{t-1}))$
- MTM => Indicador de momento = $Price_{t}-Price_{t-n}$
- ROC => Taxa de variação = $\frac{Price_{t}-Price_{t-n}}{Price_{t-n}}100$
- TSI => Indicador de Força Real = $\frac{EMA(EMA(PC,n), k)}{EMA(EMA(|PC|,n), k)}100$
    - PC => Variação do preço de fechamento = $price_{t-1} - price_t$
- K (%K) => Oscilador estocástico = $\frac{C_t-L_{t-n}}{H_{t-n}-L_{t-n}}$
    - C => Preço de fechamento
    - L => Menor valor
    - H => Maior valor
- D (%D) => Média móvel simples do Oscilador estocástico = $\frac{\sum_{i=0}^{n-1} \%K_{t-i}}{n}$
    - %K => Oscilador estocástico
- R (%R) => Williams %R = $\frac{H_{t-n}-C_t}{H_{t-n}-L_{t-n}}$ 
    - C => Preço de fechamento
    - L => Menor valor
    - H => Maior valor


#### Gerando base de dados

In [ ]:
from data_generate import Generate

dataShape, outShape = Generate(dataName, outputName)
print("Data shape: ", dataShape)
print("Output shape: ", outShape)

#### Selecionando as variáveis mais relevantes

* Randon Forest => Gera vários inputs de dados a partir de um método randomico e cria uma árvore binaria para cada um deles, realizando assim a predição com os melhores resultados (É possível treinar esse modelo e verificar quais foram as variaveis mais importantes para ele). 
* Fisher => Calcula o escore de Fisher para selecionar as caracteristicas mais importantes de um conjunto de dados, basicamente ele realiza a divizão do "erro médio quadratico ponderado" pela "variância ponderada" (Essa implementação foca em algoritmos de classificação com classes definidas por inteiros).
* Gini => calcula o índice de impureza de Gini para cada feature (variável independente) em relação à variável dependente Y. O índice de impureza de Gini é uma medida de quão impura é a divisão das amostras em duas classes, e é usado em árvores de decisão para selecionar o melhor atributo para dividir os dados em subconjuntos mais homogêneos (Utilizado em bases de dados definidas para classificação com classes referênciadas em int).
* KruskalWallis => É um teste não-paramétrico que compara as medianas de duas ou mais amostras independentes para determinar se há diferenças estatisticamente significativas entre elas (Pega os valores em que a maior próximidade estatistica -> não implica diretamente que possa ser uma boa escolha para o modelo).
* Lasso => (Least Absolute Shrinkage and Selection Operator) é um método de regressão linear que utiliza a regularização L1 para selecionar as variáveis mais importantes em um conjunto de dados. O objetivo do Lasso é minimizar a soma dos erros quadrados (Caráter linear).
* ElasticNet => é uma extensão do algoritmo Lasso (Least Absolute Shrinkage and Selection Operator) e Ridge Regression, que combina a penalização L1 e L2 para selecionar variáveis importantes em um modelo de regressão linear. É utilizado para lidar com problemas de regressão em que há um grande número de variáveis preditoras (high-dimensional data) e muitas dessas variáveis podem não ser relevantes para a predição da variável resposta (Caráter linear).
* Chi2 ($X^2$) =>  é um teste estatístico que é usado para determinar se existe uma relação significativa entre duas variáveis categóricas. Ele é usado para avaliar se as diferenças entre as frequências observadas e as frequências esperadas são significativas o suficiente para rejeitar a hipótese nula de que não há relação entre as duas variáveis.
* F_Regression => Ele utiliza a estatística F de Fisher para avaliar a relação linear entre as variáveis e calcular o p-valor associado a cada feature.
* Mutual_Info_Regression => É uma técnica de seleção de recursos que mede a dependência mútua entre cada recurso e a variável de destino, usando a entropia da informação. Ele estima a informação mútua entre cada recurso e o destino, para ajudar na seleção de recursos que são relevantes para a predição do alvo.

In [ ]:
from data_selection import Selection

d1Shape, d2Shape, outShape = Selection(dataName)
print("Dataset 1 shape: ", d1Shape)
print("Dataset 2 shape: ", d2Shape)
print("Output shape: ", outShape)

#### Separando a base de dados
* Otimização: base de dados destinada ao processo de otimização dos hiperparâmetros das redes.
* Treinamento: base de dados destinada ao processo de treinamento dos modelos de IA.
* Teste: base de dados destinada a realizar a avalição de mensurar o desempenho da proposta

In [ ]:
from data_cut import Cut
Cut(dataName, setDivision)

## Modelos de IA utilizados
* Conjunto de classificação:
    - Support Vector Machine (SVM)
    - K Neighbors Classifier (KNN)
    - Logistic Refression (LR)
* Conjunto Estatístico:
    - Generalized Autoregressive Conditional Heteroskedasticity (GARCH)
    - Autoregressive Integrated Moving Average (ARIMA)
    - Seasonal Autoregressive Integreted Moving Average (SARIMA)
* Conjunto de Regressão:
    - Long Short-Term Memory (LSTM)
    - Convolutional Neural Network (CNN)
    - Recurrent Neural Network (RNN)

#### Obtendo modelos otimizados
##### Modelos de Classificação

In [3]:
from models_classification import GetModelsClassificationOptimized
SVM, KNN, LR = GetModelsClassificationOptimized(dataName, setDivision[1])

******************** Inicio da otimização dos modelos de Classificação ********************
X_train: (2703, 4) | Y_train: (2703,)
------------------------------------ SVM ------------------------------------------
Fitting 5 folds for each of 320 candidates, totalling 1600 fits


Melhore Acurácia:  0.5475388512357089
------------------------------------ KNN -----------------------------------------
Fitting 5 folds for each of 336 candidates, totalling 1680 fits


/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/utils/_resp

Melhore Acurácia:  0.5356986376394879
------------------------------------- LR --------------------------------------------
Fitting 5 folds for each of 2400 candidates, totalling 12000 fits


/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sr-souza/.local/lib/python3.10/site-packages/sklearn/linear_mode

Melhore Acurácia:  0.5497569658382967


##### Modelos Estatísticos

In [4]:
from models_statistic import GetModelsStatisticsOptimized
import warnings
warnings.filterwarnings("ignore")

ARIMA, SARIMA, GARCH = GetModelsStatisticsOptimized(dataName, setDivision[0])


******************** Inicio da otimização dos modelos Estatisticos ********************
X_train: (225, 1) | X_test: (151, 1) | Y_train: (225, 2) | Y_test: (151, 2)
-------------------- ARIMA --------------------
------ p: 0 | d: 0 | q: 0
------ p: 0 | d: 0 | q: 1
------ p: 0 | d: 0 | q: 2
------ p: 0 | d: 1 | q: 0
------ p: 0 | d: 1 | q: 1
------ p: 0 | d: 1 | q: 2
------ p: 1 | d: 0 | q: 0
------ p: 1 | d: 0 | q: 1
------ p: 1 | d: 0 | q: 2
------ p: 1 | d: 1 | q: 0
------ p: 1 | d: 1 | q: 1
------ p: 1 | d: 1 | q: 2
------ p: 2 | d: 0 | q: 0
------ p: 2 | d: 0 | q: 1
------ p: 2 | d: 0 | q: 2
------ p: 2 | d: 1 | q: 0
------ p: 2 | d: 1 | q: 1
------ p: 2 | d: 1 | q: 2
{'autoregressive(p)': 0, 'diferencial(d)': 1, 'media(q)': 0, 'MAE': 0.0439735099337748, 'MSE': 0.004750993377483434, 'RMSE': 0.06892745010141775, 'model': <statsmodels.tsa.arima.model.ARIMAResultsWrapper object at 0x7f33f2867a60>}
-------------------- SARIMA --------------------
------ p: 0 | d: 0 | q: 0 | P: 1 | D: 0 

/home/sr-souza/.local/lib/python3.10/site-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


------ dist: t | p: 1 | q: 2
------ dist: t | p: 1 | q: 3
------ dist: t | p: 1 | q: 4
------ dist: t | p: 2 | q: 1
------ dist: t | p: 2 | q: 2
------ dist: t | p: 2 | q: 3
------ dist: t | p: 2 | q: 4


/home/sr-souza/.local/lib/python3.10/site-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


------ dist: t | p: 3 | q: 1
------ dist: t | p: 3 | q: 2


...

------ dist: studentst | p: 1 | q: 2
------ dist: studentst | p: 1 | q: 3
------ dist: studentst | p: 1 | q: 4
------ dist: studentst | p: 2 | q: 1
------ dist: studentst | p: 2 | q: 2
------ dist: studentst | p: 2 | q: 3
------ dist: studentst | p: 2 | q: 4


/home/sr-souza/.local/lib/python3.10/site-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


------ dist: studentst | p: 3 | q: 1
------ dist: studentst | p: 3 | q: 2


/home/sr-souza/.local/lib/python3.10/site-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


------ dist: studentst | p: 3 | q: 3
------ dist: studentst | p: 3 | q: 4
------ dist: studentst | p: 4 | q: 1
------ dist: studentst | p: 4 | q: 2
------ dist: studentst | p: 4 | q: 3
------ dist: studentst | p: 4 | q: 4
------ dist: ged | p: 1 | q: 1
------ dist: ged | p: 1 | q: 2
------ dist: ged | p: 1 | q: 3
------ dist: ged | p: 1 | q: 4
------ dist: ged | p: 2 | q: 1
------ dist: ged | p: 2 | q: 2
------ dist: ged | p: 2 | q: 3
------ dist: ged | p: 2 | q: 4
------ dist: ged | p: 3 | q: 1
------ dist: ged | p: 3 | q: 2
------ dist: ged | p: 3 | q: 3
------ dist: ged | p: 3 | q: 4
------ dist: ged | p: 4 | q: 1
------ dist: ged | p: 4 | q: 2
------ dist: ged | p: 4 | q: 3
------ dist: ged | p: 4 | q: 4
------ dist: gaussian | p: 1 | q: 1
------ dist: gaussian | p: 1 | q: 2
------ dist: gaussian | p: 1 | q: 3
------ dist: gaussian | p: 1 | q: 4
------ dist: gaussian | p: 2 | q: 1
------ dist: gaussian | p: 2 | q: 2
------ dist: gaussian | p: 2 | q: 3
------ dist: gaussian | p: 2 |

##### Modelos de Regressão

In [1]:
from models_regression import GetModelsRegressionOptimized
dataName    = "PETR4_B_0_60min"            # Name of the data file
setDivision = [0.1, 0.7, 0.2]              # Size of the [optimization, train, test] set
outputName  = "Fechamento"                 # Name of the output variable0

LSTM, MLP, RNN = GetModelsRegressionOptimized(dataName, setDivision[1])

Trial 60 Complete [00h 00m 44s]
val_loss: 0.6407390236854553

Best val_loss So Far: 0.37088584899902344
Total elapsed time: 00h 34m 00s
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 150)            93000     
                                                                 
 lstm_1 (LSTM)               (None, 1, 160)            199040    
                                                                 
 lstm_2 (LSTM)               (None, 1, 60)             53040     
                                                                 
 dense (Dense)               (None, 1, 1)              61        
                                                                 
Total params: 345141 (1.32 MB)
Trainable params: 345141 (1.32 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
LSTM:  None
Mode

### Recuperando Modelos já Otimizados

In [4]:
from models_classification import GetModelsClassification
from models_statistic import GetModelsStatistics
from models_regression import GetModelsRegression

SVM, KNN, LR = GetModelsClassification(dataName)
ARIMA, SARIMA, GARCH = GetModelsStatistics(dataName)
LSTM, MLP, RNN = GetModelsRegression(dataName)

In [5]:
from models_statistic import GetStatisticPredictions
from models_classification import GetClassificationPredictions
from models_regression import GetRegressionPredictions
import pandas as pd

Y_Train_statistic = pd.read_csv(f'../Data/Cut/statistic/Y/Train_{setDivision[1]}{dataName}.csv', sep=";")['OutPut |T+1|']
Y_Test_statistic  = pd.read_csv(f'../Data/Cut/statistic/Y/Test_{setDivision[2]}{dataName}.csv', sep=";")['OutPut |T+1|']

X_Train_dataset1 = pd.read_csv(f'../Data/Cut/dataset1/X/Train_{setDivision[1]}{dataName}.csv', sep=";")
Y_Train_dataset1 = pd.read_csv(f'../Data/Cut/dataset1/Y/Train_{setDivision[1]}{dataName}.csv', sep=";")['OutPut_class |T+1|']
X_Test_dataset1  = pd.read_csv(f'../Data/Cut/dataset1/X/Test_{setDivision[2]}{dataName}.csv', sep=";")
Y_Test_dataset1  = pd.read_csv(f'../Data/Cut/dataset1/Y/Test_{setDivision[2]}{dataName}.csv', sep=";")['OutPut_class |T+1|']

X_Train_dataset2 = pd.read_csv(f'../Data/Cut/dataset2/X/Train_{setDivision[1]}{dataName}.csv', sep=";")
Y_Train_dataset2 = pd.read_csv(f'../Data/Cut/dataset2/Y/Train_{setDivision[1]}{dataName}.csv', sep=";")['OutPut |T+1|']
X_Test_dataset2  = pd.read_csv(f'../Data/Cut/dataset2/X/Test_{setDivision[2]}{dataName}.csv', sep=";")
Y_Test_dataset2  = pd.read_csv(f'../Data/Cut/dataset2/Y/Test_{setDivision[2]}{dataName}.csv', sep=";")['OutPut |T+1|']


ClassificationModels = [SVM, KNN, LR]
ClassificationNames  = ['SVM', 'KNN', 'LR']
RegressionModels = [LSTM, MLP, RNN]
RegressionNames  = ['LSTM', 'MLP', 'RNN']

GetClassificationPredictions(dataName, ClassificationModels, ClassificationNames, X_Test_dataset1)
GetRegressionPredictions(dataName, RegressionNames, RegressionModels, X_Test_dataset2, Y_Test_dataset2, X_Train_dataset2, Y_Train_dataset2)
GetStatisticPredictions(dataName, Y_Train_statistic.ravel(), Y_Test_statistic.ravel(), window=100)


SVC
KNeighborsClassifier
LogisticRegression
85/85 [==============================] - 0s 2ms/step


In [6]:
from strategy import GetEnsambles
from models_buying import GetModelPrediction

GetEnsambles(dataName, setDivision[2], setDivision[1])
GetModelPrediction(dataName, setDivision[1])

Trial 100 Complete [00h 00m 18s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 30m 02s
Epoch 1/100
72/72 [==============================] - 1s 5ms/step - loss: 0.6552 - accuracy: 0.5969 - val_loss: 0.5095 - val_accuracy: 0.8990
Epoch 2/100
72/72 [==============================] - 0s 3ms/step - loss: 0.2041 - accuracy: 0.9499 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 3/100
72/72 [==============================] - 0s 2ms/step - loss: 0.0107 - accuracy: 0.9978 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 4/100
72/72 [==============================] - 0s 2ms/step - loss: 0.0056 - accuracy: 0.9987 - val_loss: 7.3615e-04 - val_accuracy: 1.0000
Epoch 5/100
72/72 [==============================] - 0s 2ms/step - loss: 0.0034 - accuracy: 0.9991 - val_loss: 5.5307e-04 - val_accuracy: 1.0000
Epoch 6/100
72/72 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 0.9991 - val_loss: 1.7997e-04 - val_accuracy: 1.0000
Epoch 7/100
72/72 [=========

In [7]:
from analyze import MakeClassificationsLogs
MakeClassificationsLogs(dataName, setDivision[2])

regressions.shape:     (676, 3)
classifications.shape: (676, 3)
statistics.shape:      (676, 3)
ensambleReg.shape:     (676,)
ensambleClass.shape:   (676,)
ensambleStat.shape:    (676,)
buying.shape:          (676,)
buying2.shape:         (676,)
outputs.shape:         (676,)
                  model  accuracy       f1 truePositives trueNegatives falsePositives falseNegatives
                   LSTM  0.505917 0.652083            29           313            307             27
                    MLP  0.507396 0.641550            45           298            291             42
                    RNN  0.513314 0.626561            71           276            265             64
                    SVM  0.507396 0.669970             5           338            331              2
                    KNN  0.470414 0.568675            82           236            254            104
                     LR  0.504438 0.669951             1           340            335              0
                 

In [8]:
from analyze_economy import GetEconomyAnalyze
GetEconomyAnalyze(dataName, setDivision[2])

Shapes dos dados importados:
Classifications:              (676, 3)
Regressions:                  (676, 3)
Statistics:                   (676, 3)
Ensamble Classification:      (676,)
Ensamble Regression:          (676,)
Ensamble Statistics:          (676,)
Buying:                       (676,)
Buying2:                      (676,)
Datas:                        (676, 14)
Output:                       (676,)
Models History:               (676, 15)
Operations History:        
                   model qtdBuying inirialValue  finalValue  percentual  accuracy       f1  truePositives  trueNegatives  falsePositives  falseNegatives
                    SVM         5         1000  575.527537  -42.447246  0.507396 0.669970            5.0          338.0           331.0             2.0
                    KNN       139         1000  591.442413  -40.855759  0.470414 0.568675           82.0          236.0           254.0           104.0
                     LR         2         1000  509.294057  -49.070